# Multi-zone Single-Duct VAV System Sizing (Design or Peak Conditions) - Take Two

In this notebook the same example as in notebook `vav_design_01.ipynb` is repeated, but now we will use the `multizone.design` module in package `hvac.air_conditioning` to design the two-zone VAV system.

In [1]:
import warnings
from hvac.fluids import CoolPropWarning
warnings.filterwarnings('ignore', category=CoolPropWarning)

In [2]:
from hvac import Quantity
from hvac.fluids import HumidAir, Fluid
from hvac.air_conditioning.multi_zone import Zone, Season
import hvac.air_conditioning.multi_zone.design as design

In [3]:
Q_ = Quantity

## Zone A 

- Enter the cooling and heating load of the zone on the peak summer and peak winter design day.
- Enter the desired state of the zone air on the peak summer and peak winter design day.
- If local exhaust of air happens in the zone, the mass flow rate can be specified through parameter `m_exhaust` of the `Season` object (zero by default).

In [4]:
zone_A = Zone(
    name='zone A',
    summer=Season(
        Q_sen=Q_(224_844, 'Btu / hr'),
        Q_lat=Q_(56_000, 'Btu / hr'),
        zone_air=HumidAir(
            Tdb=Q_(75.0, 'degF'),
            RH=Q_(50.0, 'pct')
        )
    ),
    winter=Season(
        Q_sen=Q_(-143_000, 'Btu / hr'),  # heat loss --> minus sign
        Q_lat=Q_(0.0, 'Btu / hr'),       # latent load isn't considered
        zone_air=HumidAir(
            Tdb=Q_(75.0, 'degF'),
            RH=Q_(50.0, 'pct')
        )
    )
)

## Zone B

In [5]:
zone_B = Zone(
    name='zone B',
    summer=Season(
        Q_sen=Q_(103_308, 'Btu / hr'),
        Q_lat=Q_(20_000, 'Btu / hr'),
        zone_air=HumidAir(
            Tdb=Q_(75.0, 'degF'),
            RH=Q_(50.0, 'pct')
        )
    ),
    winter=Season(
        Q_sen=Q_(49_092, 'Btu / hr'),  # zone B requires cooling in winter
        Q_lat=Q_(0.0, 'Btu / hr'),
        zone_air=HumidAir(
            Tdb=Q_(75.0, 'degF'),
            RH=Q_(50.0, 'pct')
        )
    )
)

## Configure the VAV System

- Add the zones to the system.
- Enter the state of outdoor air on the peak summer and on the peak winter design day.
- Enter the state of injected steam in the humidifier to humidify air in winter.
- Enter the required volume flow rate of outdoor air for ventilating the building.

> **IMPORTANT NOTE**<br>
> The supply fan of the VAV system is situated downstream of the cooling coil (so-called draw-through arrangement).

If humidification of supply air on the peak winter design day is necessary, saturated steam of 100 °C will be used:

In [6]:
Water = Fluid('Water')
steam = Water(T=Q_(100, 'degC'), x=Q_(1, 'frac'))

In [7]:
vav_system = design.VAVSystem(
    zones=[zone_A, zone_B],
    outdoor_air_summer=HumidAir(
        Tdb=Q_(97, 'degF'),
        Twb=Q_(76, 'degF')
    ),
    outdoor_air_winter=HumidAir(
        Tdb=Q_(7, 'degF'),
        RH=Q_(30, 'pct')
    ),
    V_vent=Q_(2400, 'ft ** 3 / min')
)

## Design the VAV System for the Peak Summer Design Day

Before running the design routines for the peak summer design day, a number of optional conditions can still be specified:
- the allowable difference between the supply and zone air temperature to ensure proper mixing (default is 12 K).
- the efficiency of the supply and/or return fan, used to determine the increase of the supply air temperature due to fan heating (ignored by default)
- the total pressure (difference or gain) of the supply and/or return fan, also used to determine the degree of fan heating (ignored by default)
- any heat gain along the supply and/or return duct (by default ignored)

In this example, a return fan is not present and duct heat gains are ignored. The fact that the efficiency and pressure gain of the supply fan are entered here may imply that a supply fan was already selected, based on a pressure loss calculation of the duct system, or that reasonable values, based on experience, were guessed for the purpose of a preliminary calculation. 

In [8]:
results = vav_system.design_summer(
    dT_sup=Q_(20, 'delta_degF'),
    eta_fan_sup=Q_(60, 'pct'),
    dP_fan_sup=Q_(3, 'inch_H2O_60F'),
    Q_duct_sup=None,
    eta_fan_ret=None,
    dP_fan_ret=None,
    Q_duct_ret=None
)

The returned results (a dictionary) are only the most principal results from the designing process:
- cooling-coil load (sensible `'Q_cc_sen'`, latent `'Q_cc_lat'` and total `'Q_cc_tot'`)
- volume flow rate `'V_sup'` of supply air (referred to the air state downstream of the supply fan)
- volume flow rate `'V_ret'` of return air (referred to the state of the return air to the AHU)
- temperature `'T_sup'` of the supply air to the zones
- temperature `'T_ret'` of the return air from the zones

However, all results can also be retrieved through instance attributes.

### Air States

In [9]:
print(
    "state of mixed air entering cooling coil = "
    f"{vav_system.summer.mixed_air.Tdb.to('degF'):~P.0f} DB, "
    f"{vav_system.summer.mixed_air.RH.to('pct'):~P.0f} RH, "
    f"{vav_system.summer.mixed_air.W.to('lb / lb'):~P.4f} lb_w / lb_a",
    "state of air leaving cooling coil = "
    f"{vav_system.summer.cooled_air.Tdb.to('degF'):~P.0f} DB, "
    f"{vav_system.summer.cooled_air.RH.to('pct'):~P.0f} RH, "
    f"{vav_system.summer.cooled_air.W.to('lb / lb'):~P.4f} lb_w / lb_a",
    "state of supply air to zones = "
    f"{vav_system.summer.supply_air.Tdb.to('degF'):~P.0f} DB, "
    f"{vav_system.summer.supply_air.RH.to('pct'):~P.0f} RH, "
    f"{vav_system.summer.supply_air.W.to('lb / lb'):~P.4f} lb_w / lb_a",
    "state of return air from zones = "
    f"{vav_system.summer.return_air.Tdb.to('degF'):~P.0f} DB, "
    f"{vav_system.summer.return_air.RH.to('pct'):~P.0f} RH, "
    f"{vav_system.summer.return_air.W.to('lb / lb'):~P.4f} lb_w / lb_a",
    "state of recirculated air to mixing chamber = "
    f"{vav_system.summer.recirculated_air.Tdb.to('degF'):~P.0f} DB, "
    f"{vav_system.summer.recirculated_air.RH.to('pct'):~P.0f} RH, "
    f"{vav_system.summer.recirculated_air.W.to('lb / lb'):~P.4f} lb_w / lb_a",
    sep='\n'
)

state of mixed air entering cooling coil = 78 °F DB, 49 % RH, 0.0101 lb_w / lb_a
state of air leaving cooling coil = 53 °F DB, 96 % RH, 0.0082 lb_w / lb_a
state of supply air to zones = 55 °F DB, 89 % RH, 0.0082 lb_w / lb_a
state of return air from zones = 75 °F DB, 50 % RH, 0.0093 lb_w / lb_a
state of recirculated air to mixing chamber = 75 °F DB, 50 % RH, 0.0093 lb_w / lb_a


### Cooling Coil
To size the cooling coil, we need to know the cooling coil load, the state of air entering the cooling coil, the state of air leaving the cooling coil, and the mass/volume flow rate of air through the coiling coil:

In [10]:
print(
    "sensible cooling coil load = "
    f"{results['Q_cc_sen'].to('Btu / hr'):~P.0f}",
    "latent cooling coil load = "
    f"{results['Q_cc_lat'].to('Btu / hr'):~P.0f}",
    "total cooling coil load = "
    f"{results['Q_cc_tot'].to('Btu / hr'):~P.0f}",
    "state of air entering cooling coil = "
    f"{vav_system.summer.mixed_air.Tdb.to('degF'):~P.0f} DB, "
    f"{vav_system.summer.mixed_air.RH.to('pct'):~P.0f} RH, "
    f"{vav_system.summer.mixed_air.W.to('lb / lb'):~P.4f} lb_w / lb_a",
    "state of air leaving cooling coil = "
    f"{vav_system.summer.cooled_air.Tdb.to('degF'):~P.0f} DB, "
    f"{vav_system.summer.cooled_air.RH.to('pct'):~P.0f} RH, "
    f"{vav_system.summer.cooled_air.W.to('lb / lb'):~P.4f} lb_w / lb_a",
    "mass flow rate of supply air = "
    f"{vav_system.summer.m_supply.to('lb / min'):~P.1f}",
    "volume flow rate of supply air = "
    f"{vav_system.summer.V_supply.to('ft ** 3 / min'):~P.1f}",
    sep='\n'
)

sensible cooling coil load = -411416 Btu/h
latent cooling coil load = -134114 Btu/h
total cooling coil load = -545782 Btu/h
state of air entering cooling coil = 78 °F DB, 49 % RH, 0.0101 lb_w / lb_a
state of air leaving cooling coil = 53 °F DB, 96 % RH, 0.0082 lb_w / lb_a
mass flow rate of supply air = 1122.5 lb/min
volume flow rate of supply air = 14750.1 ft³/min


### Supply Fan
To select a supply fan, we need to refer the volume flow rate to standard air conditions (NTP):

In [11]:
Air = Fluid('Air')
standard_air = Air(T=Q_(20, 'degC'), P=Q_(101.325, 'kPa'))
V_supply_ntp = vav_system.summer.m_supply / standard_air.rho
print(
    "volume flow rate of supply air @ NTP = "
    f"{V_supply_ntp.to('ft ** 3 / min'):~P.1f}"
)

volume flow rate of supply air @ NTP = 14926.7 ft³/min


### VAV Boxes
The required mass flow rate of supply air to each zone to meet the summer peak load and maintain the desired zone air state is needed to size the VAV-boxes: 

In [12]:
print(
    "mass flow rate of supply air to zone A = "
    f"{zone_A.summer.m_supply.to('lb / min'):~P.1f}",
    "volume flow rate of supply air to zone A = "
    f"{zone_A.summer.V_supply.to('ft ** 3 / min'):~P.1f}",
    "volume flow rate of supply air to zone A @ NTP = "
    f"{(zone_A.summer.m_supply / standard_air.rho).to('ft ** 3 / min'):~P.1f}",
    sep='\n'
)

mass flow rate of supply air to zone A = 769.1 lb/min
volume flow rate of supply air to zone A = 10106.5 ft³/min
volume flow rate of supply air to zone A @ NTP = 10227.5 ft³/min


In [13]:
print(
    "mass flow rate of supply air to zone B = "
    f"{zone_B.summer.m_supply.to('lb / min'):~P.1f}",
    "volume flow rate of supply air to zone B = "
    f"{zone_B.summer.V_supply.to('ft ** 3 / min'):~P.1f}",
    "volume flow rate of supply air to zone B @ NTP = "
    f"{(zone_B.summer.m_supply / standard_air.rho).to('ft ** 3 / min'):~P.1f}",
    sep='\n'
)

mass flow rate of supply air to zone B = 353.4 lb/min
volume flow rate of supply air to zone B = 4643.6 ft³/min
volume flow rate of supply air to zone B @ NTP = 4699.2 ft³/min


### Comparison between Desired and Resulting Air State in the Zones
The desired and the resulting state of air in zone A and in zone B can be compared. The desired state is given by `zone_air`, while the resulting state is given by `return_air`.

In [14]:
print(
    "desired state of air in zone A = "
    f"{zone_A.summer.zone_air.Tdb.to('degF'):~P.0f} DB, "
    f"{zone_A.summer.zone_air.RH.to('pct'):~P.0f} RH",
    "resulting state of air in zone A = "
    f"{zone_A.summer.return_air.Tdb.to('degF'):~P.0f} DB, "
    f"{zone_A.summer.return_air.RH.to('pct'):~P.0f} RH",
    sep='\n'
)

desired state of air in zone A = 75 °F DB, 50 % RH
resulting state of air in zone A = 75 °F DB, 50 % RH


In [15]:
print(
    "desired state of air in zone B = "
    f"{zone_B.summer.zone_air.Tdb.to('degF'):~P.0f} DB, "
    f"{zone_B.summer.zone_air.RH.to('pct'):~P.0f} RH",
    "resulting state of air in zone B = "
    f"{zone_B.summer.return_air.Tdb.to('degF'):~P.0f} DB, "
    f"{zone_B.summer.return_air.RH.to('pct'):~P.0f} RH",
    sep='\n'
)

desired state of air in zone B = 75 °F DB, 50 % RH
resulting state of air in zone B = 75 °F DB, 49 % RH


## Design the VAV System for the Peak Winter Design Day

Before running the design routines for the peak winter design day, the same optional conditions as for the peak summer design day are available regarding the additional heating of air due to operation of the supply fan and/or the return fan, and due to heat gains through ducts. Also, the maximum permissible supply air temperature to the zones can be specified to avoid stratification of the air in a zone (by default set at 40 °C).

In [16]:
results = vav_system.design_winter(
    T_sup_max=Q_(105, 'degF'),
    eta_fan_sup=Q_(60, 'pct'),
    dP_fan_sup=Q_(3, 'inch_H2O_60F'),
    Q_duct_sup=None,
    eta_fan_ret=None,
    dP_fan_ret=None,
    Q_duct_ret=None
)

The returned results (a dictionary) are:
- the peak load `'Q_ph_peak'` of the preheat-coil (i.e., the required heat rate to heat the mass flow rate of cold outdoor ventilation air to the nominal temperature at the exit of the cooling coil),
- the design load `'Q_ph'` of the preheat-coil (i.e., the required heat rate of the preheat-coil at winter design conditions),
- the mass flow rate of steam `'m_steam'` injected in the humidifier,
- the total load `'Q_cc_tot'`, the sensible load `'Q_cc_sen'`, and the latent load `'Q_cc_lat'` of the cooling coil,
- the total design load `'Q_rh_tot'` of all the reheat-coils in the zones (without diversity adjustment),
- the system volume flow rate `'V_sup'` of supply air to the zones (referred to the air state downstream of the supply fan),
- the system volume flow rate `'V_ret'` of return air from the zones (referred to the return air state),
- the temperature `'T_sup'` of the system supply air,
- the temperature `'T_ret'` of the system return air.

However, all results can also be retrieved through instance attributes.

### Air States

In [17]:
print(
    "state of mixed air entering preheat coil = "
    f"{vav_system.winter.mixed_air.Tdb.to('degF'):~P.0f} DB, "
    f"{vav_system.winter.mixed_air.RH.to('pct'):~P.0f} RH, "
    f"{vav_system.winter.mixed_air.W.to('lb / lb'):~P.4f} lb_w / lb_a",
    "state of preheated air entering humidifier = "
    f"{vav_system.winter.preheated_air.Tdb.to('degF'):~P.0f} DB, "
    f"{vav_system.winter.preheated_air.RH.to('pct'):~P.0f} RH, "
    f"{vav_system.winter.preheated_air.W.to('lb / lb'):~P.4f} lb_w / lb_a",
    "state of humidified air entering cooling coil = "
    f"{vav_system.winter.humidified_air.Tdb.to('degF'):~P.0f} DB, "
    f"{vav_system.winter.humidified_air.RH.to('pct'):~P.0f} RH, "
    f"{vav_system.winter.humidified_air.W.to('lb / lb'):~P.4f} lb_w / lb_a",
    "state of air leaving cooling coil = "
    f"{vav_system.winter.cooled_air.Tdb.to('degF'):~P.0f} DB, "
    f"{vav_system.winter.cooled_air.RH.to('pct'):~P.0f} RH, "
    f"{vav_system.winter.cooled_air.W.to('lb / lb'):~P.4f} lb_w / lb_a",
    "state of supply air to zones = "
    f"{vav_system.winter.supply_air.Tdb.to('degF'):~P.0f} DB, "
    f"{vav_system.winter.supply_air.RH.to('pct'):~P.0f} RH, "
    f"{vav_system.winter.supply_air.W.to('lb / lb'):~P.4f} lb_w / lb_a",
    "state of return air from zones = "
    f"{vav_system.winter.return_air.Tdb.to('degF'):~P.0f} DB, "
    f"{vav_system.winter.return_air.RH.to('pct'):~P.0f} RH, "
    f"{vav_system.winter.return_air.W.to('lb / lb'):~P.4f} lb_w / lb_a",
    "state of recirculated air to mixing chamber = "
    f"{vav_system.winter.recirculated_air.Tdb.to('degF'):~P.0f} DB, "
    f"{vav_system.winter.recirculated_air.RH.to('pct'):~P.0f} RH, "
    f"{vav_system.winter.recirculated_air.W.to('lb / lb'):~P.4f} lb_w / lb_a",
    sep='\n'
)

state of mixed air entering preheat coil = 54 °F DB, 73 % RH, 0.0066 lb_w / lb_a
state of preheated air entering humidifier = 57 °F DB, 66 % RH, 0.0066 lb_w / lb_a
state of humidified air entering cooling coil = 57 °F DB, 92 % RH, 0.0093 lb_w / lb_a
state of air leaving cooling coil = 57 °F DB, 92 % RH, 0.0093 lb_w / lb_a
state of supply air to zones = 59 °F DB, 86 % RH, 0.0093 lb_w / lb_a
state of return air from zones = 75 °F DB, 50 % RH, 0.0093 lb_w / lb_a
state of recirculated air to mixing chamber = 75 °F DB, 50 % RH, 0.0093 lb_w / lb_a


### Preheat Coil

In [18]:
if vav_system.winter.preheat_coil:
    print(
        "entering air = "
        f"{vav_system.winter.mixed_air.Tdb.to('degF'):~P.0f} DB, "
        f"{vav_system.winter.mixed_air.Twb.to('degF'):~P.0f} WB",
        "leaving air = "
        f"{vav_system.winter.preheated_air.Tdb.to('degF'):~P.0f} DB, "
        f"{vav_system.winter.preheated_air.Twb.to('degF'):~P.0f} WB",
        "mass flow rate of air = "
        f"{vav_system.winter.m_supply.to('lb / min'):~P.1f}",
        "design load = "
        f"{vav_system.winter.preheat_coil.Q_sen.to('Btu / hr'):~P.0f}",
        "peak load = "
        f"{vav_system.winter.Q_ph_peak.to('Btu / hr'):~P.0f}",
        sep='\n'
    )
else:
    print(
        "preheat-coil inactive"
    )

entering air = 54 °F DB, 50 °F WB
leaving air = 57 °F DB, 51 °F WB
mass flow rate of air = 673.5 lb/min
design load = 29040 Btu/h
peak load = nan Btu/h


### Humidifier

In [19]:
if vav_system.winter.humidifier:
    print(
        "entering air = "
        f"{vav_system.winter.preheated_air.Tdb.to('degF'):~P.0f} DB, "
        f"{vav_system.winter.preheated_air.Twb.to('degF'):~P.0f} WB",
        "leaving air = "
        f"{vav_system.winter.humidified_air.Tdb.to('degF'):~P.0f} DB, "
        f"{vav_system.winter.humidified_air.Twb.to('degF'):~P.0f} WB",
        "mass flow rate of air = "
        f"{vav_system.winter.m_supply.to('lb / min'):~P.1f}",
        "mass flow rate of steam = "
        f"{vav_system.winter.humidifier.m_w.to('lb / min'):~P.3f}",
        sep='\n'
    )
else:
    print("humidifier inactive")

entering air = 57 °F DB, 51 °F WB
leaving air = 57 °F DB, 56 °F WB
mass flow rate of air = 673.5 lb/min
mass flow rate of steam = 1.824 lb/min


### Cooling Coil

In [20]:
if vav_system.winter.cooling_coil:
    print(
        "entering air = "
        f"{vav_system.winter.humidified_air.Tdb.to('degF'):~P.0f} DB, "
        f"{vav_system.winter.humidified_air.Twb.to('degF'):~P.0f} WB",
        "leaving air = "
        f"{vav_system.winter.cooled_air.Tdb.to('degF'):~P.0f} DB, "
        f"{vav_system.winter.cooled_air.Twb.to('degF'):~P.0f} WB",
        "mass flow rate of air = "
        f"{vav_system.winter.m_supply.to('lb / min'):~P.1f}",
        "sensible cooling coil load = "
        f"{vav_system.winter.cooling_coil.Q_sen.to('Btu / hr'):~P.1f}",
        "latent cooling coil load = "
        f"{vav_system.winter.cooling_coil.Q_lat.to('Btu / hr'):~P.1f}",
        "total cooling coil load = "
        f"{vav_system.winter.cooling_coil.Q.to('Btu / hr'):~P.1f}",
        sep='\n'
    )
else:
    print("cooling coil inactive")

cooling coil inactive


### Supply Fan

In [21]:
V_supply_ntp = vav_system.winter.m_supply / standard_air.rho
print(
    "volume flow rate of supply air @ NTP = "
    f"{V_supply_ntp.to('ft ** 3 / min'):~P.1f}"
)

volume flow rate of supply air @ NTP = 8956.0 ft³/min


### Reheat Coils

**Zone A**

In [22]:
print(
    "entering air = "
    f"{vav_system.winter.supply_air.Tdb.to('degF'):~P.0f} DB, "
    f"{vav_system.winter.supply_air.Twb.to('degF'):~P.0f} WB",
    "leaving air = "
    f"{zone_A.winter.supply_air.Tdb.to('degF'):~P.0f} DB, "
    f"{zone_A.winter.supply_air.Twb.to('degF'):~P.0f} WB",
    "mass flow rate of air = "
    f"{zone_A.winter.m_supply.to('lb / min'):~P.1f}",
    "reheat coil load = "
    f"{zone_A.reheat_coil.Q_sen.to('Btu / hr'):~P.0f}",
    sep='\n'
)

entering air = 59 °F DB, 57 °F WB
leaving air = 96 °F DB, 69 °F WB
mass flow rate of air = 461.5 lb/min
reheat coil load = 249846 Btu/h


**Zone B**

In [23]:
print(
    "entering air = "
    f"{vav_system.winter.supply_air.Tdb.to('degF'):~P.0f} DB, "
    f"{vav_system.winter.supply_air.Twb.to('degF'):~P.0f} WB",
    "leaving air = "
    f"{zone_B.winter.supply_air.Tdb.to('degF'):~P.0f} DB, "
    f"{zone_B.winter.supply_air.Twb.to('degF'):~P.0f} WB",
    "mass flow rate of air = "
    f"{zone_B.winter.m_supply.to('lb / min'):~P.1f}",
    "reheat coil load = "
    f"{zone_B.reheat_coil.Q_sen.to('Btu / hr'):~P.0f}",
    sep='\n'
)

entering air = 59 °F DB, 57 °F WB
leaving air = 59 °F DB, 57 °F WB
mass flow rate of air = 212.0 lb/min
reheat coil load = 0 Btu/h


**Total Load of Reheat Coils**

In [24]:
print(
    "total load of reheat coils = "
    f"{vav_system.winter.Q_rh_tot.to('Btu / hr'):~P.0f}"
)

total load of reheat coils = 249846 Btu/h


### Comparison between Desired and Resulting Air State in the Zones
The desired and the resulting state of air in zone A and in zone B can be compared. The desired state is given by `zone_air`, while the resulting state is given by `return_air`.

In [25]:
print(
    "desired state of air in zone A = "
    f"{zone_A.winter.zone_air.Tdb.to('degF'):~P.0f} DB, "
    f"{zone_A.winter.zone_air.RH.to('pct'):~P.0f} RH",
    "resulting state of air in zone A = "
    f"{zone_A.winter.return_air.Tdb.to('degF'):~P.0f} DB, "
    f"{zone_A.winter.return_air.RH.to('pct'):~P.0f} RH",
    sep='\n'
)

desired state of air in zone A = 75 °F DB, 50 % RH
resulting state of air in zone A = 75 °F DB, 50 % RH


In [26]:
print(
    "desired state of air in zone B = "
    f"{zone_B.winter.zone_air.Tdb.to('degF'):~P.0f} DB, "
    f"{zone_B.winter.zone_air.RH.to('pct'):~P.0f} RH",
    "resulting state of air in zone B = "
    f"{zone_B.winter.return_air.Tdb.to('degF'):~P.0f} DB, "
    f"{zone_B.winter.return_air.RH.to('pct'):~P.0f} RH",
    sep='\n'
)

desired state of air in zone B = 75 °F DB, 50 % RH
resulting state of air in zone B = 75 °F DB, 50 % RH
